# Proyecto 1 - Etapa 2

 Estudiante 1: María Alejandra Pinzón Roncancio - 202213956

 Estudiante 2: Ignacio Chaparro - 202220577

Estudiante 3: Mateo Lopez - 202220119

# Descripción del usuario/rol de la organización que va a utilizar la aplicación
La aplicación desarrollada, que contiene las implementaciones necesarias para poder realizar las siguientes tareas específicas:
- Clasificar una o múltiples noticias ingresadas por el usuario como verdaderas o falsas, dependiendo del contenido de ellas.
- Ingresar datos ya clasificados para poder reentrenar y extender el modelo ya existente para que pueda mejorar su capacidad de clasificación. 

Tiene como público objetivo a, además de los individuos que pueden usar esta herramienta para evitar sesgos y desinformación entre las noticias que leen en línea, a las empresas de medios que pueden llegar a basarse en fuentes primarias para realizar nuevas piezas de reportaje. Para estos negocios, es imprescindible que las fuentes que utilicen para realizar sus producciones sean veraces, y por esto es que una herramienta como la que fue desarrollada es de gran utilidad. Mediante el uso de la aplicación, los usuarios de esta podrán revisar la veracidad de documentos individualmente o en grupos, y podrán extender el modelo si quieren ampliar los datos con los que se realiza el entrenamiento. 
La importancia de la existencia de la aplicación para este rol se puede evidenciar mediante el análisis de lo que el rol requiere, lo cual es generar información relevante y correcta, es por esto que una herramienta que permite corroborar con un alto grado de certeza que la información base es correcta permite que el rol en la empresa pueda funcionar mucho mejor que sin la existencia de esta.

## 1.1. Automatización del proceso de preparación de datos y documentación


### 1.1.1 

En esta sección se importan las librerías necesarias para el procesamiento de texto, manipulación de datos, modelado y creacion del pipeline.

In [22]:
import pandas as pd
import numpy as np
import unicodedata
import re
import joblib
import nltk
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FunctionTransformer

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words('spanish'))
tokenizer = WordPunctTokenizer()
stemmer = SnowballStemmer("spanish")

wpt = nltk.WordPunctTokenizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1.1.2

- Se definen las funciones que realizan la limpieza de texto:  elimina acentos, convierte a minúsculas, remueve caracteres especiales, tokeniza, elimina stopwords y aplica stemming. 

- Luego se define la función principal de limpieza, que encadena todas las anteriores para aplicarlas a un único documento textual.

- Por ultimo se realiza una clase con los metodos de fit y transform para poder integrar facilmente text_cleaning con el pipeline


In [23]:
def eliminar_acento(doc):
    return unicodedata.normalize('NFKD', doc).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def eliminar_caracteresEsp(doc):
    return re.sub(r'[^a-zA-ZñÑ\s]', '', doc)

def convertir_minuscula(doc):
    return doc.lower().strip()

def tokenizar(doc):
    return wpt.tokenize(doc)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]

def text_cleaning(doc):
    doc = eliminar_acento(doc)
    doc = eliminar_caracteresEsp(doc)
    doc = convertir_minuscula(doc)
    tokens = tokenizar(doc)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

class TextCleaner():
    def fit (self, X, y=None):
        return self
    def transform(self, X, y=None):
        return [text_cleaning(doc) for doc in X]

### 1.1.3

Antes de aplicar el preprocesamiento, se define una función para reemplazar valores nulos en los textos, lo cual previene errores en el pipeline.

In [24]:
def rellenar_nulos_texto(X):
    return X.fillna('')

### 1.1.4

Se cargan los datos preprocesados en un DataFrame, se seleccionan las variables predictoras (Titulo, Descripcion) y la variable objetivo (Label), y se dividen en conjuntos de entrenamiento(80% de los datos) y prueba(20% de los datos).

In [25]:
df = pd.read_csv('data/fake_news_spanish_limpio.csv')
X = df[['Titulo', 'Descripcion']]
y = df['Label']

# División
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 1.1.5

#### Parte 1 del pipeline: Se definen los 3 pasos del pipeline por cada columna textual:

- Relleno de nulos

- Limpieza de texto

- Vectorización TF-IDF

#### Parte 2 del pipeline: Se aplica text_pipeline a 'Titulo' y 'Descripcion'

Se aplica el text_pipeline por separado a las columnas 'Titulo' y 'Descripcion' mediante ColumnTransformer

#### Parte 3 del pipeline: Se construye el Pipeline completo 

Este contiene, el preprocesamiento de las dos columnas y la aplicacion del modelo Random Fotest seleccionado por su rendimiento en la anterior etapa.


In [26]:
text_pipeline = Pipeline([
    ('fill_na', FunctionTransformer(rellenar_nulos_texto, validate=False)),
    ('cleaner', TextCleaner()),
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2)))
])

preprocesamiento = ColumnTransformer([
    ('titulo', text_pipeline, 'Titulo'),
    ('descripcion', text_pipeline, 'Descripcion')
])

pipeline = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])


### 1.1.6
Se entrena el pipeline completo sobre el conjunto de entrenamiento.

In [27]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('titulo',
                                                  Pipeline(steps=[('fill_na',
                                                                   FunctionTransformer(func=<function rellenar_nulos_texto at 0x000001BD632E7240>)),
                                                                  ('cleaner',
                                                                   <__main__.TextCleaner object at 0x000001BD6642AB10>),
                                                                  ('tfidf',
                                                                   TfidfVectorizer(max_features=5000,
                                                                                   ngram_range=(1,
                                                                                                2)))]),
                                                  'Titulo'),
                                                 ('descripcion',
                                                  Pipeline(steps=[('fill_na',
                                                                   FunctionTransformer(func=<function rellenar_nulos_texto at 0x000001BD632E7240>)),
                                                                  ('cleaner',
                                                                   <__main__.TextCleaner object at 0x000001BD6642AB10>),
                                                                  ('tfidf',
                                                                   TfidfVectorizer(max_features=5000,
                                                                                   ngram_range=(1,
                                                                                                2)))]),
                                                  'Descripcion')])),
                ('classifier', RandomForestClassifier(random_state=42))])

### 1.1.7 
Se obtienen predicciones sobre el conjunto de prueba y se calcula el reporte de métricas 

In [28]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.88      0.91      4808
           1       0.92      0.96      0.94      6605

    accuracy                           0.93     11413
   macro avg       0.93      0.92      0.92     11413
weighted avg       0.93      0.93      0.93     11413



### 1.1.8

Se persiste el modelo con .joblib para su reutilizacion.

In [29]:
joblib.dump(pipeline, "fakenews.joblib")

['fakenews.joblib']

# 2. Acceso del modelo por medio del API

Dentro de la carpeta `api` se generó el archivo `main.py`, el cual es el punto de acceso principal a la interfaz del backend de la aplicación. En estase tienen los endpoints principales:
- `/predict` de tipo POST, el cual recibe un cuerpo en formato JSON que contiene un titulo y una descripcion de una noticia. Este endpoint puede clasificar una noticia individual como verdadera o falsa y dar un porcentaje de seguridad de la predicción.
- `/analytics` de tipo GET, la cual provee las métricas del modelo actual, incluyendo los puntajes de precisión, recall, f1-score.
- `/retrain` de tipo POST, el cual recibe un archivo de formato csv, el cual incluye las columnas `ID`,``Label``,``Titulo``,``Descripcion``,``Fecha``. Es importante que tenga la columna de Label, ya que los datos ya tienen que haber sido clasificados previamente para que el modelo pueda usar esos datos para entrenarse con los datos que previamente tenía, sumado a los nuevos. 